Import all Libraries

In [ ]:
import pandas as pd
import numpy as np
import PyPDF2
from PyPDF2 import PdfReader
from IPython.display import display

import requests 
import os 
from pyopenfec import Filing

# Bank Statement Data

In [ ]:
# Function to process PDF and return a DataFrame
def process_pdf(file_path, month):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        pdf_text = pdf_reader.getPage(0).extractText()

    data_list = [pdf_text]
    df = pd.DataFrame(data_list, columns=['Text'])
    
    output_df = pd.DataFrame(columns=['Value Type', 'Value Amount'])

    value_types = ["Beginning Balance", "Deposits / Misc Credits", "Withdrawals / Misc Debits", "Ending Balance"]

    for value_type in value_types:
        if value_type in df['Text'].values[0]:
            start_index = df['Text'].values[0].index(value_type)
            next_40_chars = df['Text'].values[0][start_index+40:start_index+55]
            output_df = output_df.append({'Value Type': value_type, 'Value Amount': next_40_chars}, ignore_index=True)

    output_df = output_df.assign(Month=month)
    cols = output_df.columns.tolist()
    cols = [cols[2]] + cols[:2] + cols[3:]
    output_df = output_df[cols]

    return output_df

# List of file paths and corresponding months
files = [
    ('C:\\Users\\scoulter\\OneDrive - AEGON\\2. 2023 Audits\\TA - PAC Audit\8 2021 Atlantic Union.pdf', 'August'),
    ('C:\\Users\\scoulter\\OneDrive - AEGON\\2. 2023 Audits\\TA - PAC Audit\9 2021 Atlantic Union.pdf', 'September')
]

output_dfs = []

# Process each file and store output DataFrames
for file_path, month in files:
    output_df = process_pdf(file_path, month)
    output_dfs.append(output_df)

# Concatenate the dataframes
bank_statement_output = pd.concat(output_dfs)

# Pivot the DataFrame
bank_statement_output = bank_statement_output.pivot(index='Month', columns='Value Type', values='Value Amount')

display(bank_statement_output)

# Payroll Data

In [ ]:
# Define function to process each file
def process_file(file_path):
    # Read the file without headers
    df = pd.read_excel(file_path, header=None)

    # Find the first row that has no missing values and store the index
    header_row = df.dropna().index[0]

    # Read in the file again using the identified row as the header
    df = pd.read_excel(file_path, header=header_row)

    # Select only required columns
    df = df[['Pay Period', 'Current Period Result']]

    # Drop the rows where either 'Pay Period' or 'Current Period Result' is null
    df = df.dropna(subset=['Pay Period', 'Current Period Result'])

    # Get the last non-null value of each column
    df = df.tail(1)
    return df

# List of file paths
file_paths = [
    'C:\\Users\\scoulter\\OneDrive - Aegon\\2. 2023 Audits\\TA - PAC Audit\\TAN PAC 060421.xlsx',
    'C:\\Users\\scoulter\\OneDrive - Aegon\\2. 2023 Audits\\TA - PAC Audit\\TAN PAC 061821.xlsx'
]

# Process each file and store result dataframes in a list
Payroll_Final_Ouput = [process_file(file_path) for file_path in file_paths]

# Concatenate the result dataframes
Payroll_Final_Ouput = pd.concat(Payroll_Final_Ouput)

# Display the result dataframe
display(Payroll_Final_Ouput)

# FEC Data

In [ ]:
##Setting up target address

base_url = 'https://api.open.fec.gov/v1' 
committee_id = 'C00236414' 

In [ ]:
##Setting the API key parameters
##API key is mine, but I think we can share
##Form type - F3X looks to me like the correct forms for monthly statements, although this can be adjusted

api_key = 'RF1B89bg3QS65X0FmvOniFhS9bnp5AT4xR72wMqo'
params = { 
    'api_key': api_key, 
    'form_type': 'F3X',
} 

In [ ]:
##Join the parameters and targets together so that the API knows where to pull information from
##Store the results in a dictionary
##Print the results (optional) to make sure the API call has worked

url = f'{base_url}/committee/{committee_id}/filings/' 
response = requests.get(url, params=params) 
results = response.json() 
print (results)

In [ ]:
##Create a new list to store the results
##Code  brings back the CSV URL and document description for each avaialble filing

data_list = []

for res in results['results']:
    data = {}
    data['document_description'] = res.get('document_description')
    data['csv_url'] = res.get('csv_url')
    data_list.append(data)

df = pd.DataFrame(data_list)
print(df)
df.to_csv('data.csv', index=False)

In [ ]:
##Create a new list to store the results
##Code  brings back the PDF URL and document description for each avaialble filing

data_list = []

for res in results['results']:
    data = {}
    data['document_description'] = res.get('document_description')
    data['pdf_url'] = res.get('pdf_url')
    data_list.append(data)

df = pd.DataFrame(data_list)
print(df)
df.to_csv('data.csv', index=False)

In [ ]:
import time
from tenacity import retry, stop_after_attempt, wait_fixed

@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def download_file(pdf_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    return requests.get(csv_url, headers=headers, stream=True)

# Assuming df is your DataFrame and it has columns 'pdf_url' and 'document_description'
for idx, row in df.iterrows():
    csv_url = row['csv_url']
    document_description = row['document_description']

    # Replaces characters in document_description that are illegal in Windows filenames
    filename = "".join(i for i in document_description if i not in ["\\", "/", ":", "*", "?", "<", ">", "|", "\""]).strip() + '.csv'

    # Path to save the pdf
    folder_path = r"C:\\Users\\scoulter\\OneDrive - AEGON\\2. 2023 Audits\TA - PAC Audit"
    file_path = os.path.join(folder_path, filename)

    # Download the PDF
    try:
        response = download_file(csv_url)

        # If the download was successful, write the file
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded {file_path}")

    except Exception as e:
        print(f"Failed to download {csv_url}. Error: {e}")

    # Delay to not overwhelm the server
    time.sleep(1)

In [ ]:
# Specify the directory containing your CSV files
directory = "C:\\Users\\scoulter\\OneDrive - AEGON\\2. 2023 Audits\TA - PAC Audit"

# Initialize an empty dataframe for storing consolidated data
FEC_Total_Receipts = pd.DataFrame(columns=['Month', 'Total Receipts'])

for filename in os.listdir(directory):
    # Only process CSV files
    if filename.endswith(".csv"):
        # Construct the full file path
        filepath = os.path.join(directory, filename)
        try:
            # Use pandas to read the CSV file, specify the column of interest and the first row
            df = pd.read_csv(filepath, header=None, skiprows=1, nrows=1, usecols=[23])
            total_receipts = df.iloc[0,0]
            # Remove spaces in filename, replace with _, and remove the extension
            month = filename.replace(" ", "_")[:-4]
            # Append the month and total_receipts to the consolidated dataframe
            FEC_Total_Receipts = FEC_Total_Receipts.append({'Month': month, 'Total Receipts': total_receipts}, ignore_index=True)
        except Exception as e:
            print(f"An error occurred with file {filename}: {str(e)}")
            
display(FEC_Total_Receipts)


In [ ]:
# Create a new DataFrame from the split 'Month' column
split_df = FEC_Total_Receipts['Month'].str.split('_', expand=True)

# If there is more than one column in split_df, rename them and concatenate them to the original DataFrame
if split_df.shape[1] > 1:
    # Naming the new columns
    split_df.columns = ['Month', 'Info', 'Year']
    # Dropping the original 'Month' column
    FEC_Total_Receipts = FEC_Total_Receipts.drop(columns=['Month'])
    # Concatenating the split_df to the original DataFrame
    FEC_Total_Receipts = pd.concat([FEC_Total_Receipts, split_df], axis=1)
else:
    # If there's only one column, it means there were no underscores in 'Month'
    FEC_Total_Receipts['Month'] = split_df[0]

display(FEC_Total_Receipts)


In [48]:
# Filter the data for the year 2022
FEC_Receipts_2022 = FEC_Total_Receipts[FEC_Total_Receipts['Year'] == '2022']

FEC_Receipts_2022['Month'] = FEC_Receipts_2022['Month'].str.capitalize()

# Merge the filtered data with bank_statement_output on the 'Month' column
Test_One = pd.merge(bank_statement_output, FEC_Receipts_2022, on='Month', how='outer')

display(Test_One)

C:\Anaconda3201910\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Month,Beginning Balance,Deposits / Misc Credits,Ending Balance,Withdrawals / Misc Debits,Total Receipts,Info,Year
0,August,"229,231.19","21,011.55","231,735.93","18,506.81",18123.75,MONTHLY,2022
1,September,"231,735.93","20,867.39","237,598.23","15,005.09",17886.58,MONTHLY,2022
2,April,NaN,NaN,NaN,NaN,27141.62,MONTHLY,2022
3,February,NaN,NaN,NaN,NaN,19543.70,MONTHLY,2022
4,July,NaN,NaN,NaN,NaN,18719.98,MONTHLY,2022
5,June,NaN,NaN,NaN,NaN,18730.81,MONTHLY,2022
6,March,NaN,NaN,NaN,NaN,19849.95,MONTHLY,2022
7,May,NaN,NaN,NaN,NaN,18920.80,MONTHLY,2022
8,October,NaN,NaN,NaN,NaN,17775.65,MONTHLY,2022
